# Orthomosaic-Basemap Feature Matching and Registration

This notebook performs feature matching between orthomosaics (with/without GCPs) and a ground control basemap using SIFT and evaluates 2D shifting and registration.

## Goals:
1. **Feature Matching**: Use SIFT to find corresponding features between orthos and basemap
2. **Multi-Resolution Analysis**: Evaluate matching at full, half, and quarter resolution
3. **2D Registration**: Apply computed shifts to register orthos to basemap
4. **Visualization**: Create visualizations of matches and registered orthos

## Inputs:
- **Ground Control Basemap**: `TestsiteNewWest_Spexigeo_RTK.tiff`
- **Orthomosaic (No GCPs)**: `outputs/orthomosaics/orthomosaic_no_gcps.tif`
- **Orthomosaic (With GCPs)**: `outputs/orthomosaics/orthomosaic_with_gcps.tif`

## Outputs:
- All outputs saved to `outputs/test_matching/`
- Match visualizations at different resolutions
- Registered orthomosaics


## Setup: Install Dependencies


In [1]:
# Install required packages if needed
import subprocess
import sys

packages = ['opencv-python', 'scikit-image', 'rasterio', 'numpy', 'matplotlib', 'pillow', 'scipy']
for package in packages:
    try:
        if package == 'opencv-python':
            __import__('cv2')
        elif package == 'scikit-image':
            __import__('skimage')
        elif package == 'pillow':
            __import__('PIL')
        else:
            __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("✓ Dependencies installed")


✓ Dependencies installed


## Imports


In [2]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling, transform_bounds
from rasterio import Affine
import cv2
from PIL import Image
import warnings
import json
from typing import Dict, Tuple, Optional, List
import logging

warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

print("✓ Imports successful!")


✓ Imports successful!


## Step 1: Setup Paths and Output Directories


In [3]:
# Define paths
data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
output_dir = Path("outputs")

# Input files
basemap_path = data_dir / "Michael_RTK_orthos" / "TestsiteNewWest_Spexigeo_RTK.tiff"
ortho_no_gcps_path = output_dir / "orthomosaics" / "orthomosaic_no_gcps.tif"
ortho_with_gcps_path = output_dir / "orthomosaics" / "orthomosaic_with_gcps.tif"

# Output directory structure
matching_output_dir = output_dir / "test_matching"
matching_output_dir.mkdir(parents=True, exist_ok=True)

# Subdirectories
reprojected_dir = matching_output_dir / "reprojected"
reprojected_dir.mkdir(exist_ok=True)

converted_dir = matching_output_dir / "converted"
converted_dir.mkdir(exist_ok=True)

matches_dir = matching_output_dir / "matches"
matches_dir.mkdir(exist_ok=True)

registered_dir = matching_output_dir / "registered"
registered_dir.mkdir(exist_ok=True)

print(f"✓ Output directory: {matching_output_dir}")
print(f"  - Reprojected: {reprojected_dir}")
print(f"  - Converted: {converted_dir}")
print(f"  - Matches: {matches_dir}")
print(f"  - Registered: {registered_dir}")


✓ Output directory: outputs/test_matching
  - Reprojected: outputs/test_matching/reprojected
  - Converted: outputs/test_matching/converted
  - Matches: outputs/test_matching/matches
  - Registered: outputs/test_matching/registered


## Step 2: Load and Check Input Files


In [4]:
# Check if files exist
if not basemap_path.exists():
    raise FileNotFoundError(f"Basemap not found: {basemap_path}")
if not ortho_no_gcps_path.exists():
    raise FileNotFoundError(f"Orthomosaic (no GCPs) not found: {ortho_no_gcps_path}")
if not ortho_with_gcps_path.exists():
    raise FileNotFoundError(f"Orthomosaic (with GCPs) not found: {ortho_with_gcps_path}")

print("✓ All input files found")

# Get basic info about each file
print("\n📊 File Information:")
for name, path in [("Basemap", basemap_path), ("Ortho (No GCPs)", ortho_no_gcps_path), ("Ortho (With GCPs)", ortho_with_gcps_path)]:
    with rasterio.open(path) as src:
        file_size_mb = path.stat().st_size / (1024 * 1024)
        print(f"\n{name}:")
        print(f"  Path: {path}")
        print(f"  Size: {file_size_mb:.2f} MB")
        print(f"  CRS: {src.crs}")
        print(f"  Dimensions: {src.width} x {src.height}")
        print(f"  Bands: {src.count}")
        print(f"  Bounds: {src.bounds}")
        if src.crs:
            pixel_size_x = abs(src.transform[0])
            pixel_size_y = abs(src.transform[4])
            print(f"  Pixel size: {pixel_size_x:.4f} m (X), {pixel_size_y:.4f} m (Y)")


✓ All input files found

📊 File Information:

Basemap:
  Path: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/Michael_RTK_orthos/TestsiteNewWest_Spexigeo_RTK.tiff
  Size: 19804.91 MB
  CRS: EPSG:32610
  Dimensions: 90129 x 90188
  Bands: 4
  Bounds: BoundingBox(left=506424.37839793676, bottom=5450017.622213458, right=507501.0951215451, top=5451095.043774429)
  Pixel size: 0.0119 m (X), 0.0119 m (Y)

Ortho (No GCPs):
  Path: outputs/orthomosaics/orthomosaic_no_gcps.tif
  Size: 3887.28 MB
  CRS: EPSG:4326
  Dimensions: 38531 x 39277
  Bands: 4
  Bounds: BoundingBox(left=-122.91170031378732, bottom=49.20303508811748, right=-122.89720111440005, top=49.212718829461046)
  Pixel size: 0.0000 m (X), 0.0000 m (Y)

Ortho (With GCPs):
  Path: outputs/orthomosaics/orthomosaic_with_gcps.tif
  Size: 3867.91 MB
  CRS: EPSG:4326
  Dimensions: 38538 x 39233
  Bands: 4
  Bounds: BoundingBox(left=-122.9117167927641, bottom=49.203044960109175, right=-122.89720605758104, top=49.21272

## Step 3: Reproject Orthomosaics to Match Basemap CRS


In [5]:
# Get basemap CRS (target CRS)
with rasterio.open(basemap_path) as basemap_src:
    target_crs = basemap_src.crs
    target_bounds = basemap_src.bounds
    target_transform = basemap_src.transform
    target_width = basemap_src.width
    target_height = basemap_src.height

print(f"Target CRS: {target_crs}")
print(f"Target bounds: {target_bounds}")
print(f"Target dimensions: {target_width} x {target_height}")

# Reproject each orthomosaic
ortho_paths = {
    'no_gcps': ortho_no_gcps_path,
    'with_gcps': ortho_with_gcps_path
}

reprojected_paths = {}

for ortho_name, ortho_path in ortho_paths.items():
    print(f"\n{'='*60}")
    print(f"Reprojecting {ortho_name}...")
    
    reprojected_path = reprojected_dir / f"{ortho_name}_reprojected.tif"
    
    # Check if already reprojected (skip if file exists and is valid)
    if reprojected_path.exists():
        try:
            # Verify the file is valid by checking its properties
            with rasterio.open(reprojected_path) as check_src:
                if check_src.crs == target_crs:
                    print(f"  ✓ Already reprojected: {reprojected_path}")
                    print(f"    Dimensions: {check_src.width}x{check_src.height}, CRS: {check_src.crs}")
                    reprojected_paths[ortho_name] = reprojected_path
                    continue
                else:
                    print(f"  ⚠️  Existing file has wrong CRS, will reproject...")
        except Exception as e:
            print(f"  ⚠️  Existing file is invalid ({e}), will reproject...")
    
    with rasterio.open(ortho_path) as ortho_src:
        source_crs = ortho_src.crs
        source_bounds = ortho_src.bounds
        
        print(f"  Source CRS: {source_crs}")
        print(f"  Source bounds: {source_bounds}")
        
        # Check if reprojection is needed
        if source_crs == target_crs:
            print(f"  ✓ Already in target CRS, copying...")
            import shutil
            shutil.copy(ortho_path, reprojected_path)
            reprojected_paths[ortho_name] = reprojected_path
            continue
        
        # Calculate transform using source bounds (not target bounds)
        # First, transform source bounds to target CRS to get output extent
        try:
            print(f"  Transforming source bounds to target CRS...")
            src_bounds_target_crs = transform_bounds(
                source_crs, target_crs,
                source_bounds.left, source_bounds.bottom,
                source_bounds.right, source_bounds.top
            )
            
            print(f"  Source bounds in target CRS: {src_bounds_target_crs}")
            
            # Get target pixel size
            target_pixel_size_x = abs(target_transform[0])
            target_pixel_size_y = abs(target_transform[4])
            
            # Use intersection of bounds (where both images overlap)
            output_left = max(src_bounds_target_crs[0], target_bounds.left)
            output_bottom = max(src_bounds_target_crs[1], target_bounds.bottom)
            output_right = min(src_bounds_target_crs[2], target_bounds.right)
            output_top = min(src_bounds_target_crs[3], target_bounds.top)
            
            print(f"  Output bounds (intersection): left={output_left:.2f}, bottom={output_bottom:.2f}, right={output_right:.2f}, top={output_top:.2f}")
            
            # Validate bounds
            if output_right <= output_left or output_top <= output_bottom:
                raise ValueError(f"Invalid output bounds: width={output_right-output_left}, height={output_top-output_bottom}")
            
            # Calculate dimensions using target pixel size
            width = int((output_right - output_left) / target_pixel_size_x)
            height = int((output_top - output_bottom) / target_pixel_size_y)
            
            # Validate dimensions
            if width <= 0 or height <= 0:
                raise ValueError(f"Invalid dimensions: width={width}, height={height}")
            
            # Create transform for output
            transform = Affine.translation(output_left, output_top) * Affine.scale(target_pixel_size_x, -target_pixel_size_y)
            
            print(f"  ✓ Transform calculated: {width}x{height} pixels")
            
        except Exception as e:
            print(f"  ❌ Transform calculation failed: {e}")
            print(f"  Trying alternative approach using source resolution...")
            
            # Alternative: use source resolution and calculate output bounds
            try:
                # Get source pixel size
                source_pixel_size_x = abs(ortho_src.transform[0])
                source_pixel_size_y = abs(ortho_src.transform[4])
                
                # Transform source bounds to target CRS
                src_bounds_target_crs = transform_bounds(
                    source_crs, target_crs,
                    source_bounds.left, source_bounds.bottom,
                    source_bounds.right, source_bounds.top
                )
                
                # Use source pixel size (approximate)
                output_left = src_bounds_target_crs[0]
                output_bottom = src_bounds_target_crs[1]
                output_right = src_bounds_target_crs[2]
                output_top = src_bounds_target_crs[3]
                
                # Calculate dimensions
                width = int((output_right - output_left) / source_pixel_size_x)
                height = int((output_top - output_bottom) / source_pixel_size_y)
                
                transform = Affine.translation(output_left, output_top) * Affine.scale(source_pixel_size_x, -source_pixel_size_y)
                
                print(f"  ✓ Using source resolution approach: {width}x{height} pixels")
                
            except Exception as e2:
                print(f"  ❌ Alternative approach also failed: {e2}")
                print(f"  Skipping {ortho_name}...")
                continue
        
        # Reproject
        print(f"  Reprojecting to {width}x{height}...")
        
        # Read source data
        source_data = ortho_src.read()
        source_count = ortho_src.count
        
        # Reproject each band
        reprojected_data = np.zeros((source_count, height, width), dtype=source_data.dtype)
        
        for band_idx in range(1, source_count + 1):
            reproject(
                source=rasterio.band(ortho_src, band_idx),
                destination=reprojected_data[band_idx - 1],
                src_transform=ortho_src.transform,
                src_crs=source_crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.bilinear
            )
        
        # Save reprojected file with LZW compression
        # Note: We skip saving JPEG here because PIL has a 65500 pixel limit
        # JPEG conversion will be done in Step 4 at different resolutions
        
        # Save GeoTIFF with compression
        with rasterio.open(
            reprojected_path,
            'w',
            driver='GTiff',
            height=height,
            width=width,
            count=source_count,
            dtype=reprojected_data.dtype,
            crs=target_crs,
            transform=transform,
            compress='lzw',
            BIGTIFF='YES',
            tiled=True,
            blockxsize=512,
            blockysize=512,
            predictor=2  # Horizontal differencing for better compression
        ) as dst:
            dst.write(reprojected_data)
        
        print(f"  ✓ Saved: {reprojected_path}")
        print(f"  Note: JPEG conversion will be done in Step 4 (PIL limit: 65500 pixels)")
        reprojected_paths[ortho_name] = reprojected_path

print(f"\n✓ Reprojection complete!")


Target CRS: EPSG:32610
Target bounds: BoundingBox(left=506424.37839793676, bottom=5450017.622213458, right=507501.0951215451, top=5451095.043774429)
Target dimensions: 90129 x 90188

Reprojecting no_gcps...
  ✓ Already reprojected: outputs/test_matching/reprojected/no_gcps_reprojected.tif
    Dimensions: 88515x89120, CRS: EPSG:32610

Reprojecting with_gcps...
  ✓ Already reprojected: outputs/test_matching/reprojected/with_gcps_reprojected.tif
    Dimensions: 88586x89028, CRS: EPSG:32610

✓ Reprojection complete!


In [6]:
# Ensure imports are available
try:
    from pathlib import Path
    import numpy as np
    import rasterio
    from rasterio import Affine
    from PIL import Image
# Increase PIL image size limit to handle large images
    import cv2
    from typing import Tuple, Dict
except (NameError, ImportError):
    # Re-import if not available
    from pathlib import Path
    import numpy as np
    import rasterio
    from rasterio import Affine
    import cv2
    from typing import Tuple, Dict

Image.MAX_IMAGE_PIXELS = None  # Disable decompression bomb protection

def convert_geotiff_to_jpeg(geotiff_path: Path, output_path: Path, downsample_factor: float = 1.0) -> Tuple[np.ndarray, Dict]:
    """
    Convert GeoTIFF to JPEG format at specified resolution.
    
    Args:
        geotiff_path: Path to input GeoTIFF
        output_path: Path to save JPEG
        downsample_factor: Factor to downsample (1.0 = full res, 0.5 = half, 0.25 = quarter)
        
    Returns:
        Tuple of (image array, metadata dict with transform info)
    """
    with rasterio.open(geotiff_path) as src:
        # Get metadata
        metadata = {
            'transform': src.transform,
            'crs': src.crs,
            'bounds': src.bounds,
            'width': src.width,
            'height': src.height
        }
        
        # Calculate output dimensions
        if downsample_factor < 1.0:
            new_height = int(src.height * downsample_factor)
            new_width = int(src.width * downsample_factor)
        else:
            new_height = src.height
            new_width = src.width
        
        print(f"    Processing {src.width}x{src.height} -> {new_width}x{new_height} (factor: {downsample_factor})")
        
        # For very large images, process in tiles to avoid memory issues
        # Use tile-based processing for images > 10k pixels
        use_tiles = src.width > 10000 or src.height > 10000
        
        if use_tiles:
            # Process in tiles with downsampling
            print(f"    Using tile-based processing (tile size: 2048)...")
            
            # Compute global statistics for normalization
            # Use a sample-based approach for memory efficiency
            print(f"    Computing image statistics for normalization...")
            
            # Read a representative sample (center region) to estimate min/max
            sample_size = min(5000, src.width, src.height)
            center_x = src.width // 2
            center_y = src.height // 2
            sample_x = max(0, center_x - sample_size // 2)
            sample_y = max(0, center_y - sample_size // 2)
            sample_w = min(sample_size, src.width - sample_x)
            sample_h = min(sample_size, src.height - sample_y)
            
            window = rasterio.windows.Window(sample_x, sample_y, sample_w, sample_h)
            
            if src.count >= 3:
                sample_data = src.read([1, 2, 3], window=window)  # Shape: (3, H, W)
                data_min = sample_data.min(axis=(1, 2), keepdims=True)  # Shape: (3, 1, 1)
                data_max = sample_data.max(axis=(1, 2), keepdims=True)  # Shape: (3, 1, 1)
            else:
                sample_data = src.read(1, window=window)  # Shape: (H, W)
                data_min_val = sample_data.min()
                data_max_val = sample_data.max()
                data_min = np.array([[[data_min_val]]])  # Shape: (1, 1, 1)
                data_max = np.array([[[data_max_val]]])  # Shape: (1, 1, 1)
                # Expand to 3 channels
                data_min = np.repeat(data_min, 3, axis=0)  # Shape: (3, 1, 1)
                data_max = np.repeat(data_max, 3, axis=0)  # Shape: (3, 1, 1)
            
            data_range = data_max - data_min
            data_range[data_range == 0] = 1
            
            # Create output array
            output_array = np.zeros((new_height, new_width, 3), dtype=np.uint8)
            tile_size = 2048
            
            # Process in tiles
            num_tiles_x = (src.width + tile_size - 1) // tile_size
            num_tiles_y = (src.height + tile_size - 1) // tile_size
            
            for tile_y in range(num_tiles_y):
                for tile_x in range(num_tiles_x):
                    # Calculate tile window
                    col_off = tile_x * tile_size
                    row_off = tile_y * tile_size
                    width = min(tile_size, src.width - col_off)
                    height = min(tile_size, src.height - row_off)
                    
                    window = rasterio.windows.Window(col_off, row_off, width, height)
                    
                    # Read tile
                    if src.count >= 3:
                        tile_data = src.read([1, 2, 3], window=window)
                    else:
                        tile_data = src.read(1, window=window)
                        tile_data = np.stack([tile_data] * 3)
                    
                    # Normalize
                    tile_normalized = ((tile_data - data_min) / data_range * 255).astype(np.uint8)
                    
                    # Ensure tile_normalized has correct shape (3, H, W)
                    if len(tile_normalized.shape) == 2:
                        # Single band, convert to 3 channels
                        tile_normalized = np.stack([tile_normalized] * 3)
                    elif tile_normalized.shape[0] != 3:
                        # Wrong number of channels, fix it
                        if tile_normalized.shape[0] == 1:
                            tile_normalized = np.repeat(tile_normalized, 3, axis=0)
                        else:
                            tile_normalized = tile_normalized[:3]  # Take first 3 channels
                    
                    # Downsample tile
                    if downsample_factor < 1.0:
                        tile_h, tile_w = tile_normalized.shape[1], tile_normalized.shape[2]
                        new_tile_h = int(tile_h * downsample_factor)
                        new_tile_w = int(tile_w * downsample_factor)
                        
                        if new_tile_h > 0 and new_tile_w > 0:
                            tile_resized = np.zeros((3, new_tile_h, new_tile_w), dtype=np.uint8)
                            for i in range(3):
                                tile_resized[i] = cv2.resize(tile_normalized[i], (new_tile_w, new_tile_h), interpolation=cv2.INTER_AREA)
                            tile_normalized = tile_resized
                    
                    # Calculate output position
                    out_col_off = int(col_off * downsample_factor)
                    out_row_off = int(row_off * downsample_factor)
                    tile_h, tile_w = tile_normalized.shape[1], tile_normalized.shape[2]
                    
                    # Ensure we don't exceed output bounds
                    out_h = min(tile_h, new_height - out_row_off)
                    out_w = min(tile_w, new_width - out_col_off)
                    
                    if out_h > 0 and out_w > 0 and out_row_off >= 0 and out_col_off >= 0:
                        # Ensure we don't exceed tile dimensions
                        slice_h = min(out_h, tile_h)
                        slice_w = min(out_w, tile_w)
                        
                        # Verify tile_normalized has correct shape
                        if len(tile_normalized.shape) == 3 and tile_normalized.shape[0] == 3:
                            # Slice tile to fit output bounds, then convert to (H, W, C) and place in output
                            tile_slice = tile_normalized[:, :slice_h, :slice_w]  # Shape: (3, slice_h, slice_w)
                            tile_rgb = tile_slice.transpose(1, 2, 0)  # Shape: (slice_h, slice_w, 3)
                            # Place in output array (use actual slice dimensions)
                            output_array[out_row_off:out_row_off+slice_h, out_col_off:out_col_off+slice_w] = tile_rgb
                        else:
                            print(f"      Warning: Unexpected tile shape {tile_normalized.shape}, skipping tile")
                
                if (tile_y + 1) % 10 == 0:
                    print(f"      Processed {tile_y + 1}/{num_tiles_y} tile rows...")
            
            img_array = output_array
            
        else:
            # For smaller images, read normally
            if src.count >= 3:
                data = src.read([1, 2, 3])
            else:
                data = src.read(1)
                data = np.stack([data, data, data])
            
            # Normalize to 0-255
            if data.dtype != np.uint8:
                data_min = data.min(axis=(1, 2), keepdims=True)
                data_max = data.max(axis=(1, 2), keepdims=True)
                data_range = data_max - data_min
                data_range[data_range == 0] = 1
                data = ((data - data_min) / data_range * 255).astype(np.uint8)
            
            # Downsample if needed
            if downsample_factor < 1.0:
                height, width = data.shape[1], data.shape[2]
                new_height = int(height * downsample_factor)
                new_width = int(width * downsample_factor)
                
                data_resized = np.zeros((3, new_height, new_width), dtype=np.uint8)
                for i in range(3):
                    data_resized[i] = cv2.resize(data[i], (new_width, new_height), interpolation=cv2.INTER_AREA)
                data = data_resized
            
            # Convert to PIL Image format (H, W, C)
            img_array = data.transpose(1, 2, 0)
        
        # Update metadata
        metadata['width'] = new_width
        metadata['height'] = new_height
        if downsample_factor < 1.0:
            old_transform = metadata['transform']
            metadata['transform'] = Affine(
                old_transform[0] / downsample_factor, old_transform[1], old_transform[2],
                old_transform[3], old_transform[4] / downsample_factor, old_transform[5]
            )
        
        # Save as JPEG (only if within PIL limits) or PNG
        # Validate array before saving
        if img_array.dtype != np.uint8:
            print(f"    ⚠️  Converting array from {img_array.dtype} to uint8...")
            img_array = np.clip(img_array, 0, 255).astype(np.uint8)
        
        if len(img_array.shape) != 3 or img_array.shape[2] != 3:
            raise ValueError(f"Invalid image array shape: {img_array.shape}, expected (H, W, 3)")
        
        try:
            img = Image.fromarray(img_array, 'RGB')
            
            if new_width <= 65500 and new_height <= 65500:
                # Save as JPEG
                img.save(output_path, 'JPEG', quality=95, optimize=True)
                
                # Verify file was written correctly
                if output_path.exists() and output_path.stat().st_size > 0:
                    print(f"    ✓ Saved JPEG: {output_path} ({new_width}x{new_height}, {output_path.stat().st_size / 1024 / 1024:.2f} MB)")
                else:
                    raise IOError(f"JPEG file was not written correctly: {output_path}")
            else:
                # Save as PNG for very large images (PNG has no dimension limit)
                print(f"    ⚠️  Image too large for JPEG (PIL limit: 65500), saving as PNG...")
                output_path_png = output_path.with_suffix('.png')
                img.save(output_path_png, 'PNG', compress_level=6)
                
                # Verify file was written correctly
                if output_path_png.exists() and output_path_png.stat().st_size > 0:
                    print(f"    ✓ Saved PNG: {output_path_png} ({new_width}x{new_height}, {output_path_png.stat().st_size / 1024 / 1024:.2f} MB)")
                    output_path = output_path_png
                else:
                    raise IOError(f"PNG file was not written correctly: {output_path_png}")
                    
        except Exception as e:
            print(f"    ❌ Error saving image: {e}")
            raise
        
        return img_array, metadata

# Check if required variables are defined
try:
    from pathlib import Path
except ImportError:
    from pathlib import Path

try:
    _ = converted_dir
except NameError:
    try:
        _ = output_dir
    except NameError:
        output_dir = Path("outputs")
    converted_dir = output_dir / "test_matching" / "converted"
    converted_dir.mkdir(parents=True, exist_ok=True)
    print(f"ℹ️  converted_dir not defined, using default: {converted_dir}")

try:
    _ = basemap_path
except NameError:
    try:
        _ = data_dir
    except NameError:
        data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
    basemap_path = data_dir / "Michael_RTK_orthos" / "TestsiteNewWest_Spexigeo_RTK.tiff"
    if not basemap_path.exists():
        raise FileNotFoundError(f"Basemap not found: {basemap_path}")

try:
    _ = reprojected_paths
except NameError:
    try:
        _ = reprojected_dir
    except NameError:
        try:
            _ = output_dir
        except NameError:
            output_dir = Path("outputs")
        reprojected_dir = output_dir / "test_matching" / "reprojected"
    
    # Try to find reprojected files
    reprojected_paths = {}
    for ortho_name in ['no_gcps', 'with_gcps']:
        reproj_path = reprojected_dir / f"{ortho_name}_reprojected.tif"
        if reproj_path.exists():
            reprojected_paths[ortho_name] = reproj_path
            print(f"ℹ️  Found existing reprojected file: {reproj_path}")
        else:
            print(f"⚠️  Reprojected file not found: {reproj_path}")
            print(f"   Please run Step 3 first to create reprojected files")

# Convert basemap and reprojected orthos at different resolutions
# Skip full resolution for now (too large, causes kernel crashes)
# Can enable later if needed
resolutions = {
    # 'full': 1.0,  # Skipped - too large for memory
    'half': 0.5,
    'quarter': 0.25
}

converted_files = {}

print("Converting files to JPEG at different resolutions...")

# Convert basemap at all resolutions
for res_name, factor in resolutions.items():
    basemap_jpeg = converted_dir / f"basemap_{res_name}.jpg"
    if not basemap_jpeg.exists():
        print(f"\nConverting basemap at {res_name} resolution...")
        basemap_img, basemap_meta = convert_geotiff_to_jpeg(basemap_path, basemap_jpeg, downsample_factor=factor)
        if res_name not in converted_files:
            converted_files[res_name] = {}
        converted_files[res_name]['basemap'] = {'path': basemap_jpeg, 'img': basemap_img, 'meta': basemap_meta}
    else:
        # Check if PNG was saved instead (for very large images)
        basemap_png = basemap_jpeg.with_suffix('.png')
        if basemap_png.exists():
            print(f"\nBasemap PNG ({res_name}) already exists: {basemap_png}")
            basemap_img = np.array(Image.open(basemap_png))
            basemap_jpeg = basemap_png  # Update path
        else:
            print(f"\nBasemap JPEG ({res_name}) already exists: {basemap_jpeg}")
            basemap_img = np.array(Image.open(basemap_jpeg))
        with rasterio.open(basemap_path) as src:
            basemap_meta = {
                'transform': src.transform,
                'crs': src.crs,
                'bounds': src.bounds,
                'width': basemap_img.shape[1],
                'height': basemap_img.shape[0]
            }
        if res_name not in converted_files:
            converted_files[res_name] = {}
        converted_files[res_name]['basemap'] = {'path': basemap_jpeg, 'img': basemap_img, 'meta': basemap_meta}

# Convert reprojected orthos at all resolutions
for ortho_name, reprojected_path in reprojected_paths.items():
    for res_name, factor in resolutions.items():
        ortho_jpeg = converted_dir / f"{ortho_name}_{res_name}.jpg"
        if not ortho_jpeg.exists():
            print(f"\nConverting {ortho_name} at {res_name} resolution...")
            ortho_img, ortho_meta = convert_geotiff_to_jpeg(reprojected_path, ortho_jpeg, downsample_factor=factor)
            converted_files[res_name][ortho_name] = {'path': ortho_jpeg, 'img': ortho_img, 'meta': ortho_meta}
        else:
            # Check if PNG was saved instead (for very large images)
            ortho_png = ortho_jpeg.with_suffix('.png')
            if ortho_png.exists():
                print(f"\n{ortho_name} PNG ({res_name}) already exists: {ortho_png}")
                ortho_img = np.array(Image.open(ortho_png))
                ortho_jpeg = ortho_png  # Update path
            else:
                print(f"\n{ortho_name} JPEG ({res_name}) already exists: {ortho_jpeg}")
                ortho_img = np.array(Image.open(ortho_jpeg))
            with rasterio.open(reprojected_path) as src:
                ortho_meta = {
                    'transform': src.transform,
                    'crs': src.crs,
                    'bounds': src.bounds,
                    'width': ortho_img.shape[1],
                    'height': ortho_img.shape[0]
                }
            if res_name not in converted_files:
                converted_files[res_name] = {}
            converted_files[res_name][ortho_name] = {'path': ortho_jpeg, 'img': ortho_img, 'meta': ortho_meta}

print(f"\n✓ Conversion complete!")


Converting files to JPEG at different resolutions...

Basemap JPEG (half) already exists: outputs/test_matching/converted/basemap_half.jpg

Basemap JPEG (quarter) already exists: outputs/test_matching/converted/basemap_quarter.jpg

no_gcps JPEG (half) already exists: outputs/test_matching/converted/no_gcps_half.jpg

no_gcps JPEG (quarter) already exists: outputs/test_matching/converted/no_gcps_quarter.jpg

with_gcps JPEG (half) already exists: outputs/test_matching/converted/with_gcps_half.jpg

with_gcps JPEG (quarter) already exists: outputs/test_matching/converted/with_gcps_quarter.jpg

✓ Conversion complete!


In [7]:
def perform_tile_based_orb_matching(img1: np.ndarray, img2: np.ndarray, tile_size: int = 4000, overlap: int = 800, max_features: int = 2000) -> Dict:
    """
    Perform ORB feature matching using tile-based approach for memory efficiency.
    
    Args:
        img1: First image (numpy array)
        img2: Second image (numpy array)
        tile_size: Size of each tile in pixels (default: 4000)
        overlap: Overlap between tiles in pixels (default: 800)
        max_features: Maximum features per tile (default: 2000)
        
    Returns:
        Dictionary with matching results aggregated from all tiles
    """
    # Convert to grayscale if needed
    if len(img1.shape) == 3:
        gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    else:
        gray1 = img1
    
    if len(img2.shape) == 3:
        gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    else:
        gray2 = img2
    
    h1, w1 = gray1.shape

    # First, try coarse alignment using phase correlation
    # This helps if images are significantly offset
    try:
        from scipy import signal
        # Downsample for phase correlation (much faster)
        ds_factor = 8  # Downsample by 8x for speed
        gray1_small = cv2.resize(gray1, (w1//ds_factor, h1//ds_factor), interpolation=cv2.INTER_AREA)
        gray2_small = cv2.resize(gray2, (w2//ds_factor, h2//ds_factor), interpolation=cv2.INTER_AREA)
        
        # Find overlap region (use smaller image as template)
        h_small, w_small = min(gray1_small.shape[0], gray2_small.shape[0]), min(gray1_small.shape[1], gray2_small.shape[1])
        template = gray1_small[:h_small, :w_small]
        search = gray2_small
        
        # Phase correlation to find offset
        correlation = signal.correlate2d(search, template, mode='full')
        y_peak, x_peak = np.unravel_index(np.argmax(correlation), correlation.shape)
        
        # Calculate offset (account for correlation padding)
        coarse_offset_y = (y_peak - search.shape[0] + 1) * ds_factor
        coarse_offset_x = (x_peak - search.shape[1] + 1) * ds_factor
        
        print(f"    Coarse alignment: offset_x={coarse_offset_x:.1f}, offset_y={coarse_offset_y:.1f} px")
        
        # Use this offset to constrain tile matching
        use_coarse_alignment = True
        alignment_tolerance = max(w1, h1) * 0.1  # 10% of image size
        
    except Exception as e:
        print(f"    ⚠️  Coarse alignment failed: {e}, proceeding without it")
        use_coarse_alignment = False
        coarse_offset_x = 0
        coarse_offset_y = 0
    h2, w2 = gray2.shape
    
    # Calculate number of tiles
    step = tile_size - overlap
    tiles_x1 = max(1, (w1 + step - 1) // step)
    tiles_y1 = max(1, (h1 + step - 1) // step)
    tiles_x2 = max(1, (w2 + step - 1) // step)
    tiles_y2 = max(1, (h2 + step - 1) // step)
    
    print(f"    Processing {tiles_y1}x{tiles_x1} tiles for img1, {tiles_y2}x{tiles_x2} tiles for img2")
    
    all_offsets_x = []
    all_offsets_y = []
    all_matches = []
    total_keypoints1 = 0
    total_keypoints2 = 0
    
    # Process tiles from img1
    for ty1 in range(tiles_y1):
        for tx1 in range(tiles_x1):
            y1_start = ty1 * step
            y1_end = min(y1_start + tile_size, h1)
            x1_start = tx1 * step
            x1_end = min(x1_start + tile_size, w1)
            
            tile1 = gray1[y1_start:y1_end, x1_start:x1_end]
            
            # Find corresponding region in img2
            best_match_count = 0
            best_offset_x = None
            best_offset_y = None
            
            for ty2 in range(tiles_y2):
                for tx2 in range(tiles_x2):
                    y2_start = ty2 * step
                    y2_end = min(y2_start + tile_size, h2)
                    x2_start = tx2 * step
                    x2_end = min(x2_start + tile_size, w2)
                    
                    tile2 = gray2[y2_start:y2_end, x2_start:x2_end]
                    
                    # Use ORB for faster, more memory-efficient matching
                    orb = cv2.ORB_create(nfeatures=max_features, edgeThreshold=31, patchSize=31)
                    kp1_tile, des1_tile = orb.detectAndCompute(tile1, None)
                    kp2_tile, des2_tile = orb.detectAndCompute(tile2, None)
                    
                    if des1_tile is None or des2_tile is None or len(des1_tile) == 0 or len(des2_tile) == 0:
                        continue
                    
                    # ORB uses binary descriptors - use Hamming distance matcher
                    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
                    
                    try:
                        matches = bf.knnMatch(des1_tile, des2_tile, k=2)
                    except:
                        continue
                    
                    # Apply Lowe's ratio test
                    good_matches = []
                    for match_pair in matches:
                        if len(match_pair) == 2:
                            m, n = match_pair
                            if m.distance < 0.7 * n.distance:
                                good_matches.append(m)
                    
                    if len(good_matches) >= 4:
                        # Calculate offset for this tile pair
                        src_pts = np.float32([kp1_tile[m.queryIdx].pt for m in good_matches])
                        dst_pts = np.float32([kp2_tile[m.trainIdx].pt for m in good_matches])
                        
                        # Calculate mean offset
                        offsets = dst_pts - src_pts
                        mean_offset_x = float(np.mean(offsets[:, 0]))
                        mean_offset_y = float(np.mean(offsets[:, 1]))
                        
                        # Adjust for tile positions
                        global_offset_x = mean_offset_x + (x2_start - x1_start)
                        global_offset_y = mean_offset_y + (y2_start - y1_start)
                        
                        if len(good_matches) > best_match_count:
                            best_match_count = len(good_matches)
                            best_offset_x = global_offset_x
                            best_offset_y = global_offset_y
                    
                    total_keypoints1 += len(kp1_tile) if kp1_tile else 0
                    total_keypoints2 += len(kp2_tile) if kp2_tile else 0
            
            if best_offset_x is not None:
                all_offsets_x.append(best_offset_x)
                all_offsets_y.append(best_offset_y)
                all_matches.append(best_match_count)
    
    # Aggregate results
    if len(all_offsets_x) > 0:
        # Use median offset (more robust than mean)
        offset_x = float(np.median(all_offsets_x))
        offset_y = float(np.median(all_offsets_y))
        total_matches = sum(all_matches)
        
        # Calculate RMSE from offsets
        if len(all_offsets_x) > 1:
            errors_x = np.array(all_offsets_x) - offset_x
            errors_y = np.array(all_offsets_y) - offset_y
            rmse_2d = float(np.sqrt(np.mean(errors_x**2 + errors_y**2)))
        else:
            rmse_2d = 0.0
    else:
        offset_x = None
        offset_y = None
        total_matches = 0
        rmse_2d = None
    
    return {
        'num_keypoints1': total_keypoints1,
        'num_keypoints2': total_keypoints2,
        'num_matches': total_matches,
        'num_inliers': total_matches,  # All matches are considered inliers
        'offset_x': offset_x,
        'offset_y': offset_y,
        'rmse_2d': rmse_2d,
        'homography': None,  # Not computed for tile-based
        'confidence': total_matches / max(total_keypoints1, total_keypoints2) if total_keypoints1 > 0 and total_keypoints2 > 0 else 0.0,
        'good_matches': [],  # Not stored for tile-based
        'kp1': [],  # Not stored for tile-based
        'kp2': []  # Not stored for tile-based
    }


In [8]:
## Step 5: Perform Feature Matching

# Perform matching at different resolutions
# Use pre-saved JPEG files at each resolution
# Note: Only quarter resolution (half skipped due to memory)
resolutions = {
    # 'full': 1.0,  # Skipped - too large for memory
    # 'half': 0.5,  # Skipped - too large, causes kernel crashes
    'quarter': 0.25
}

# Ensure required variables and imports are available
try:
    import numpy as np
    from PIL import Image
    from pathlib import Path
    import cv2
    from typing import Dict
except (NameError, ImportError):
    import numpy as np
    from PIL import Image
    from pathlib import Path
    import cv2
    from typing import Dict

Image.MAX_IMAGE_PIXELS = None  # Disable decompression bomb protection

# Ensure converted_dir is defined
try:
    _ = converted_dir
except NameError:
    try:
        _ = output_dir
    except NameError:
        output_dir = Path("outputs")
    converted_dir = output_dir / "test_matching" / "converted"
    print(f"ℹ️  converted_dir not defined, using default: {converted_dir}")

# Ensure matches_dir is defined
try:
    _ = matches_dir
except NameError:
    try:
        _ = output_dir
    except NameError:
        output_dir = Path("outputs")
    matches_dir = output_dir / "test_matching" / "matches"
    matches_dir.mkdir(parents=True, exist_ok=True)
    print(f"ℹ️  matches_dir not defined, using default: {matches_dir}")

matching_results = {}

for ortho_name in ['no_gcps', 'with_gcps']:
    print(f"\n{'='*60}")
    print(f"Matching {ortho_name} to basemap")
    print(f"{'='*60}")
    
    matching_results[ortho_name] = {}
    
    for res_name, factor in resolutions.items():
        print(f"\n📊 Resolution: {res_name} (factor: {factor})")
        
        # Load pre-saved images at this resolution
        # Check if converted_files is available, otherwise load from disk
        try:
            _ = converted_files
        except NameError:
            print("  ⚠️  converted_files not defined, loading from disk...")
            converted_files = {}
        
        # Load basemap image
        if res_name in converted_files and 'basemap' in converted_files[res_name]:
            basemap_img = converted_files[res_name]['basemap']['img']
        else:
            # Load from disk
            basemap_jpeg = converted_dir / f"basemap_{res_name}.jpg"
            basemap_png = converted_dir / f"basemap_{res_name}.png"
            if basemap_png.exists():
                basemap_img = np.array(Image.open(basemap_png))
            elif basemap_jpeg.exists():
                basemap_img = np.array(Image.open(basemap_jpeg))
            else:
                print(f"  ❌ Basemap image not found for {res_name} resolution")
                continue
        
        # Load ortho image
        if res_name in converted_files and ortho_name in converted_files[res_name]:
            ortho_img = converted_files[res_name][ortho_name]['img']
        else:
            # Load from disk
            ortho_jpeg = converted_dir / f"{ortho_name}_{res_name}.jpg"
            ortho_png = converted_dir / f"{ortho_name}_{res_name}.png"
            if ortho_png.exists():
                ortho_img = np.array(Image.open(ortho_png))
            elif ortho_jpeg.exists():
                ortho_img = np.array(Image.open(ortho_jpeg))
            else:
                print(f"  ❌ Ortho image not found for {ortho_name} at {res_name} resolution")
                continue
        
        print(f"  Basemap: {basemap_img.shape[1]}x{basemap_img.shape[0]}")
        print(f"  Ortho: {ortho_img.shape[1]}x{ortho_img.shape[0]}")
        
        # Perform matching using tile-based ORB
        result = perform_tile_based_orb_matching(ortho_img, basemap_img, tile_size=4000, overlap=800, max_features=2000)
        
        # Clean up memory
        import gc
        del basemap_img, ortho_img
        gc.collect()
        
        print(f"  Keypoints (ortho): {result['num_keypoints1']}")
        print(f"  Keypoints (basemap): {result['num_keypoints2']}")
        print(f"  Matches: {result['num_matches']}")
        print(f"  Inliers: {result['num_inliers']}")
        
        if result['offset_x'] is not None:
            print(f"  Offset X: {result['offset_x']:.2f} px")
            print(f"  Offset Y: {result['offset_y']:.2f} px")
            print(f"  RMSE 2D: {result['rmse_2d']:.2f} px")
            print(f"  Confidence: {result['confidence']:.3f}")
            
            # Scale offsets back to full resolution
            result['offset_x_full'] = result['offset_x'] / factor
            result['offset_y_full'] = result['offset_y'] / factor
            result['rmse_2d_full'] = result['rmse_2d'] / factor if result['rmse_2d'] else None
        else:
            print(f"  ⚠️  Matching failed: {result.get('error', 'Unknown error')}")
        
        matching_results[ortho_name][res_name] = result

print(f"\n✓ Feature matching complete!")

# Save matching results to JSON file
import json
try:
    _ = matches_dir
except NameError:
    try:
        _ = output_dir
    except NameError:
        output_dir = Path("outputs")
    matches_dir = output_dir / "test_matching" / "matches"
    matches_dir.mkdir(parents=True, exist_ok=True)

matching_results_file = matches_dir / "matching_results.json"

# Convert numpy types to native Python types for JSON serialization
def convert_to_native_types(obj):
    """Recursively convert numpy types to native Python types."""
    import numpy as np
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_native_types(value) for key, value in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_native_types(item) for item in obj]
    return obj

matching_results_serializable = convert_to_native_types(matching_results)
with open(matching_results_file, 'w') as f:
    json.dump(matching_results_serializable, f, indent=2)
print(f"✓ Saved matching results to: {matching_results_file}")



Matching no_gcps to basemap

📊 Resolution: quarter (factor: 0.25)
  Basemap: 22532x22547
  Ortho: 22128x22280
    Processing 7x7 tiles for img1, 8x8 tiles for img2
  Keypoints (ortho): 4015488
  Keypoints (basemap): 3709650
  Matches: 650
  Inliers: 650
  Offset X: 255.66 px
  Offset Y: 200.45 px
  RMSE 2D: 10797.95 px
  Confidence: 0.000

Matching with_gcps to basemap

📊 Resolution: quarter (factor: 0.25)
  Basemap: 22532x22547
  Ortho: 22146x22257
    Processing 7x7 tiles for img1, 8x8 tiles for img2
  Keypoints (ortho): 3949920
  Keypoints (basemap): 3709650
  Matches: 647
  Inliers: 647
  Offset X: 449.58 px
  Offset Y: 87.06 px
  RMSE 2D: 8986.82 px
  Confidence: 0.000

✓ Feature matching complete!


In [1]:
# Create summary visualization
# Ensure imports are available
try:
    import matplotlib.pyplot as plt
    from pathlib import Path
except (NameError, ImportError):
    import matplotlib.pyplot as plt
    from pathlib import Path

# Ensure required variables are defined
try:
    _ = matching_results
except NameError:
    print("⚠️  matching_results not defined. Attempting to load from JSON file...")
    try:
        import json
        from pathlib import Path
        try:
            _ = matches_dir
        except NameError:
            try:
                _ = output_dir
            except NameError:
                output_dir = Path("outputs")
            matches_dir = output_dir / "test_matching" / "matches"
        matching_results_file = matches_dir / "matching_results.json"
        if matching_results_file.exists():
            with open(matching_results_file, 'r') as f:
                matching_results = json.load(f)
            print(f"✓ Loaded matching results from: {matching_results_file}")
        else:
            print(f"   JSON file not found: {matching_results_file}")
            print("   Please run Step 5 (matching loop) first.")
            matching_results = {}
    except Exception as e:
        print(f"   Error loading JSON: {e}")
        matching_results = {}
    print("   Skipping visualization...")
    matching_results = {}

try:
    _ = resolutions
except NameError:
    resolutions = {'quarter': 0.25}  # Default

try:
    _ = matches_dir
except NameError:
    try:
        _ = output_dir
    except NameError:
        output_dir = Path("outputs")
    matches_dir = output_dir / "test_matching" / "matches"
    matches_dir.mkdir(parents=True, exist_ok=True)

# Determine subplot dimensions dynamically
# Calculate max resolutions dynamically
max_resolutions = 1
for on in ['no_gcps', 'with_gcps']:
    if on in matching_results:
        num_res = len(matching_results[on].keys())
        if num_res > max_resolutions:
            max_resolutions = num_res

# Check if we have any results to visualize
has_results = False
if matching_results:
    for on in ['no_gcps', 'with_gcps']:
        if on in matching_results and matching_results[on]:
            has_results = True
            break

if not has_results:
    print("⚠️  No matching results available. Please run Step 5 (matching loop) first.")
    fig, axes = plt.subplots(2, max_resolutions, figsize=(6*max_resolutions, 12))
    if max_resolutions == 1:
        axes = axes.reshape(2, 1)  # Ensure 2D array for indexing
    fig.suptitle('SIFT Feature Matching Results at Different Resolutions', fontsize=16, fontweight='bold')

for ortho_idx, ortho_name in enumerate(['no_gcps', 'with_gcps']):
    # Get available resolutions for this ortho
    available_resolutions = list(matching_results[ortho_name].keys()) if ortho_name in matching_results else []
    for res_idx, res_name in enumerate(available_resolutions):
        if res_name not in matching_results[ortho_name]:
            continue
        # Get factor for this resolution
        factor = resolutions.get(res_name, 0.25)  # Default to quarter if not found
        ax = axes[ortho_idx, res_idx]
        
        result = matching_results[ortho_name][res_name]
        
        if result['num_matches'] > 0 and result['offset_x'] is not None:
            # Load visualization if it exists
            vis_path = matches_dir / f"{ortho_name}_{res_name}_matches.jpg"
            if vis_path.exists():
                vis_img = plt.imread(vis_path)
                ax.imshow(vis_img)
                ax.axis('off')
                
                title = f"{ortho_name.replace('_', ' ').title()} - {res_name.title()}\n"
                title += f"Matches: {result['num_matches']}, "
                title += f"Offset: ({result['offset_x']:.1f}, {result['offset_y']:.1f}) px"
                ax.set_title(title, fontsize=10)
            else:
                ax.text(0.5, 0.5, f"No visualization\navailable", 
                       ha='center', va='center', transform=ax.transAxes)
                ax.set_title(f"{ortho_name} - {res_name}", fontsize=10)
        else:
            ax.text(0.5, 0.5, f"No matches found\n{result.get('error', '')}", 
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f"{ortho_name} - {res_name}", fontsize=10)
            ax.axis('off')

plt.tight_layout()
summary_vis_path = matches_dir / "matching_summary.png"
plt.savefig(summary_vis_path, dpi=150, bbox_inches='tight')
plt.close()

print(f"✓ Summary visualization saved: {summary_vis_path}")

# Print summary table
print("\n" + "="*60)
print("Matching Results Summary")
print("="*60)

for ortho_name in ['no_gcps', 'with_gcps']:
    print(f"\n{ortho_name.replace('_', ' ').title()}:")
    print(f"  {'Resolution':<12} {'Matches':<10} {'Offset X':<12} {'Offset Y':<12} {'RMSE 2D':<12} {'Confidence':<10}")
    print(f"  {'-'*12} {'-'*10} {'-'*12} {'-'*12} {'-'*12} {'-'*10}")
    
    # Only show resolutions that were actually processed
    available_resolutions = list(matching_results[ortho_name].keys()) if ortho_name in matching_results else []
    for res_name in available_resolutions:
        result = matching_results[ortho_name][res_name]
        matches = result['num_matches']
        offset_x = f"{result['offset_x']:.2f}" if result['offset_x'] is not None else "N/A"
        offset_y = f"{result['offset_y']:.2f}" if result['offset_y'] is not None else "N/A"
        rmse = f"{result['rmse_2d']:.2f}" if result['rmse_2d'] is not None else "N/A"
        conf = f"{result['confidence']:.3f}" if result['confidence'] else "0.000"
        
        print(f"  {res_name:<12} {matches:<10} {offset_x:<12} {offset_y:<12} {rmse:<12} {conf:<10}")


⚠️  matching_results not defined. Attempting to load from JSON file...
   JSON file not found: outputs/test_matching/matches/matching_results.json
   Please run Step 5 (matching loop) first.
   Skipping visualization...
⚠️  No matching results available. Please run Step 5 (matching loop) first.
✓ Summary visualization saved: outputs/test_matching/matches/matching_summary.png

Matching Results Summary

No Gcps:
  Resolution   Matches    Offset X     Offset Y     RMSE 2D      Confidence
  ------------ ---------- ------------ ------------ ------------ ----------

With Gcps:
  Resolution   Matches    Offset X     Offset Y     RMSE 2D      Confidence
  ------------ ---------- ------------ ------------ ------------ ----------


## Step 7: Apply 2D Shift and Register Orthomosaics


In [ ]:
# Ensure imports are available
try:
    import numpy as np
    from typing import Tuple
except (NameError, ImportError):
    import numpy as np
    from typing import Tuple

def apply_2d_shift_to_image(img: np.ndarray, shift_x: float, shift_y: float) -> np.ndarray:
    """
    Apply 2D shift to an image.
    
    Args:
        img: Input image array
        shift_x: Shift in X direction (pixels)
        shift_y: Shift in Y direction (pixels)
        
    Returns:
        Shifted image
    """
    try:
        from scipy.ndimage import shift
        use_scipy = True
    except ImportError:
        use_scipy = False
    
    if use_scipy:
        if len(img.shape) == 3:
            # RGB image
            shifted = np.zeros_like(img)
            for i in range(img.shape[2]):
                shifted[:, :, i] = shift(img[:, :, i], (shift_y, shift_x), mode='constant', cval=0, order=1)
        else:
            # Grayscale
            shifted = shift(img, (shift_y, shift_x), mode='constant', cval=0, order=1)
    else:
        # Fallback: integer shift using numpy
        shift_x_int = int(round(shift_x))
        shift_y_int = int(round(shift_y))
        
        if len(img.shape) == 3:
            shifted = np.zeros_like(img)
            h, w = img.shape[:2]
            if shift_y_int >= 0 and shift_x_int >= 0:
                shifted[shift_y_int:, shift_x_int:] = img[:h-shift_y_int, :w-shift_x_int]
            elif shift_y_int < 0 and shift_x_int < 0:
                shifted[:h+shift_y_int, :w+shift_x_int] = img[-shift_y_int:, -shift_x_int:]
            else:
                # Handle other cases
                if shift_y_int >= 0:
                    shifted[shift_y_int:, :] = img[:h-shift_y_int, :]
                if shift_x_int >= 0:
                    shifted[:, shift_x_int:] = img[:, :w-shift_x_int]
        else:
            shifted = np.zeros_like(img)
            h, w = img.shape
            if shift_y_int >= 0 and shift_x_int >= 0:
                shifted[shift_y_int:, shift_x_int:] = img[:h-shift_y_int, :w-shift_x_int]
            elif shift_y_int < 0 and shift_x_int < 0:
                shifted[:h+shift_y_int, :w+shift_x_int] = img[-shift_y_int:, -shift_x_int:]
    
    return shifted

# Choose best resolution for registration (prefer full res if available, otherwise use best match count)
registered_orthos = {}

for ortho_name in ['no_gcps', 'with_gcps']:
    print(f"\n{'='*60}")
    print(f"Registering {ortho_name}...")
    print(f"{'='*60}")
    
    # Find best resolution (prefer full, then highest match count)
    best_res = None
    best_matches = 0
    
    # Use available resolutions from matching_results
    available_resolutions = [r for r in matching_results[ortho_name].keys()]
    for res_name in available_resolutions:
        result = matching_results[ortho_name][res_name]
        if result['num_matches'] > best_matches and result['offset_x'] is not None:
            best_matches = result['num_matches']
            best_res = res_name
    
    if best_res is None:
        print(f"  ⚠️  No valid matches found for {ortho_name}")
        continue
    
    result = matching_results[ortho_name][best_res]
    print(f"  Using {best_res} resolution results (matches: {result['num_matches']})")
    
    # Get offset scaled to the resolution we'll use for registration
    # Scale offsets back to the resolution we're using (half or quarter)
    if resolution_used_for_registration == 'half':
        scale_factor = 0.5
    else:  # quarter
        scale_factor = 0.25
    
    # The offsets from matching are already at the matching resolution
    # Scale them to the registration resolution
    shift_x = result['offset_x'] / scale_factor if result['offset_x'] is not None else 0
    shift_y = result['offset_y'] / scale_factor if result['offset_y'] is not None else 0
    
    print(f"  Applying shift: X={shift_x:.2f} px, Y={shift_y:.2f} px")
    
    # Load full resolution image
    # Load highest available resolution image (prefer half, fallback to quarter)
    # Full resolution skipped due to memory constraints
    if 'half' in converted_files and ortho_name in converted_files['half']:
        ortho_img = converted_files['half'][ortho_name]['img']
        resolution_used_for_registration = 'half'
    elif 'quarter' in converted_files and ortho_name in converted_files['quarter']:
        ortho_img = converted_files['quarter'][ortho_name]['img']
        resolution_used_for_registration = 'quarter'
    else:
        print(f"  ⚠️  No converted images found for {ortho_name}")
        continue
    # Apply shift
    shifted_img = apply_2d_shift_to_image(ortho_img, shift_x, shift_y)
    
    # Save registered image
    registered_path = registered_dir / f"{ortho_name}_registered.jpg"
    
    if len(shifted_img.shape) == 3:
        img_pil = Image.fromarray(shifted_img)
    else:
        img_pil = Image.fromarray(shifted_img).convert('RGB')
    
    img_pil.save(registered_path, 'JPEG', quality=95)
    print(f"  ✓ Saved registered image: {registered_path}")
    
    registered_orthos[ortho_name] = {
        'path': registered_path,
        'img': shifted_img,
        'shift_x': shift_x,
        'shift_y': shift_y,
        'resolution_used': best_res
    }

print(f"\n✓ Registration complete!")


## Step 8: Create Final Visualizations


In [ ]:
# Ensure imports are available
try:
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    from pathlib import Path
except (NameError, ImportError):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    from pathlib import Path

# Create side-by-side comparison: original vs registered
for ortho_name in ['no_gcps', 'with_gcps']:
    if ortho_name not in registered_orthos:
        continue
    
    print(f"\nCreating visualization for {ortho_name}...")
    
    # Load images (use full resolution)
    # Load images (use highest available resolution - half or quarter)
    if 'half' in converted_files and ortho_name in converted_files['half']:
        resolution_key = 'half'
    elif 'quarter' in converted_files and ortho_name in converted_files['quarter']:
        resolution_key = 'quarter'
    else:
        print(f"  ⚠️  No converted images found for {ortho_name}, skipping visualization")
        continue
    
    original_img = converted_files[resolution_key][ortho_name]['img']
    registered_img = registered_orthos[ortho_name]['img']
    basemap_img = converted_files[resolution_key]['basemap']['img']
    
    # Resize to same size for comparison (use smallest)
    min_h = min(original_img.shape[0], registered_img.shape[0], basemap_img.shape[0])
    min_w = min(original_img.shape[1], registered_img.shape[1], basemap_img.shape[1])
    
    original_resized = cv2.resize(original_img, (min_w, min_h))
    registered_resized = cv2.resize(registered_img, (min_w, min_h))
    basemap_resized = cv2.resize(basemap_img, (min_w, min_h))
    
    # Create comparison figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 16))
    fig.suptitle(f'Registration Results: {ortho_name.replace("_", " ").title()}', fontsize=16, fontweight='bold')
    
    # Basemap (ground truth)
    axes[0, 0].imshow(basemap_resized)
    axes[0, 0].set_title('Ground Truth Basemap', fontweight='bold')
    axes[0, 0].axis('off')
    
    # Original ortho
    axes[0, 1].imshow(original_resized)
    axes[0, 1].set_title('Original Orthomosaic', fontweight='bold')
    axes[0, 1].axis('off')
    
    # Registered ortho
    axes[1, 0].imshow(registered_resized)
    shift_info = registered_orthos[ortho_name]
    title = f"Registered Orthomosaic\n"
    title += f"Shift: ({shift_info['shift_x']:.2f}, {shift_info['shift_y']:.2f}) px\n"
    title += f"Resolution: {shift_info['resolution_used']}"
    axes[1, 0].set_title(title, fontweight='bold')
    axes[1, 0].axis('off')
    
    # Difference map
    diff = np.abs(registered_resized.astype(float) - basemap_resized.astype(float))
    diff_norm = (diff / diff.max() * 255).astype(np.uint8) if diff.max() > 0 else diff.astype(np.uint8)
    axes[1, 1].imshow(diff_norm, cmap='hot')
    axes[1, 1].set_title('Difference Map (Registered vs Basemap)', fontweight='bold')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    
    comparison_path = registered_dir / f"{ortho_name}_comparison.png"
    plt.savefig(comparison_path, dpi=150, bbox_inches='tight')
    plt.close()
    
    print(f"  ✓ Saved comparison: {comparison_path}")

# Save matching results to JSON
results_json_path = matching_output_dir / "matching_results.json"

def convert_to_native(obj):
    """Convert numpy types to native Python types for JSON."""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native(item) for item in obj]
    elif hasattr(obj, '__dict__'):
        # Handle OpenCV keypoints (skip them for JSON)
        return None
    return obj

# Clean results for JSON (remove OpenCV objects)
json_results = {}
for ortho_name in matching_results:
    json_results[ortho_name] = {}
    for res_name in matching_results[ortho_name]:
        result = matching_results[ortho_name][res_name].copy()
        # Remove OpenCV objects
        result.pop('kp1', None)
        result.pop('kp2', None)
        result.pop('good_matches', None)
        json_results[ortho_name][res_name] = convert_to_native(result)

with open(results_json_path, 'w') as f:
    json.dump(json_results, f, indent=2)

print(f"\n✓ Results saved to: {results_json_path}")
print(f"\n{'='*60}")
print("✓ Feature Matching and Registration Complete!")
print(f"{'='*60}")
